In [19]:
from nemo.collections.tts.models import T5TTS_Model
from nemo.collections.tts.data.text_to_speech_dataset import T5TTSDataset, DatasetSample
from omegaconf.omegaconf import OmegaConf, open_dict
import torch
import os
import soundfile as sf
from IPython.display import display, Audio
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [42]:
# hparams_file = "/Data/T5Plus_Ckpts_New/continuouscheckpoints_ks3ks3/multiencoder_small_sp_ks3_hparams.yaml"
# checkpoint_file = "/Data/T5Plus_Ckpts_New/continuouscheckpoints_ks3ks3/multiencoder_small_sp_ks3_epoch302.ckpt"
# checkpoint_file = "/Data/T5Plus_Ckpts_New/continuouscheckpoints_ks3ks3/dpo_checkpoints/multiencoder_small_sp_ks3_epoch302_v3_beta0.05_dpo_epoch6.ckpt"

hparams_file = "/Data/T5Plus_Ckpts_New/continuouscheckpoints_ks3ks3/decodercontext_small_sp_ks3Correct_hparams.yaml"
checkpoint_file = "/Data/T5Plus_Ckpts_New/continuouscheckpoints_ks3ks3/dpo_checkpoints/decodercontext_small_sp_ks3Correct_epoch305_v3_beta0.05_epoch9.ckpt"
out_dir = "/Data/NotebookGenerations"
codecmodel_path = "/Data/Checkpoints/AudioCodec_21Hz_no_eliz.nemo"

In [43]:
model_cfg = OmegaConf.load(hparams_file).cfg

with open_dict(model_cfg):
    model_cfg.codecmodel_path = codecmodel_path
    if hasattr(model_cfg, 'text_tokenizer'):
        # Backward compatibility for models trained with absolute paths in text_tokenizer
        model_cfg.text_tokenizer.g2p.phoneme_dict = "scripts/tts_dataset_files/ipa_cmudict-0.7b_nv23.01.txt"
        model_cfg.text_tokenizer.g2p.heteronyms = "scripts/tts_dataset_files/heteronyms-052722"
        model_cfg.text_tokenizer.g2p.phoneme_probability = 1.0
    model_cfg.train_ds = None
    model_cfg.validation_ds = None


model = T5TTS_Model(cfg=model_cfg)
# Load weights from checkpoint file
print("Loading weights from checkpoint")
ckpt = torch.load(checkpoint_file)
model.load_state_dict(ckpt['state_dict'])
print("Loaded weights.")

if model_cfg.t5_decoder.pos_emb == "learnable":
    if (model_cfg.t5_decoder.use_flash_self_attention) is False and (model_cfg.t5_decoder.use_flash_self_attention is False):
        print("Using kv cache for inference.")
        model.use_kv_cache_for_inference = True

model.cuda()
model.eval()

[NeMo W 2025-01-13 23:43:38 experimental:26] `<class 'nemo.collections.tts.g2p.models.i18n_ipa.IpaG2p'>` is experimental and not ready for production yet. Use at your own risk.
[NeMo W 2025-01-13 23:43:39 i18n_ipa:124] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2025-01-13 23:43:39 experimental:26] `<class 'nemo.collections.common.tokenizers.text_to_speech.tts_tokenizers.IPATokenizer'>` is experimental and not ready for production yet. Use at your own risk.
[NeMo W 2025-01-13 23:43:39 experimental:26] `<class 'nemo.collections.tts.g2p.models.i18n_ipa.IpaG2p'>` is experimental and not ready for production yet. Use at your own risk.
[NeMo W 2025-01-13 23:43:39 i18n_ipa:124] apply_to_oov_word=None, This means that some of words will remain unchanged if they 

[NeMo I 2025-01-13 23:43:46 audio_codec:94] Vector quantizer does not support commit loss.
[NeMo I 2025-01-13 23:43:47 features:305] PADDING: 1
[NeMo I 2025-01-13 23:43:47 features:305] PADDING: 1
[NeMo I 2025-01-13 23:43:47 features:305] PADDING: 1
[NeMo I 2025-01-13 23:43:47 features:305] PADDING: 1
[NeMo I 2025-01-13 23:43:47 features:305] PADDING: 1
[NeMo I 2025-01-13 23:43:47 features:305] PADDING: 1
[NeMo I 2025-01-13 23:43:47 features:305] PADDING: 1
[NeMo I 2025-01-13 23:43:47 save_restore_connector:275] Model AudioCodecModel was successfully restored from /Data/Checkpoints/AudioCodec_21Hz_no_eliz.nemo.
Loading weights from checkpoint
Loaded weights.


T5TTS_Model(
  (audio_embeddings): ModuleList(
    (0-7): 8 x Embedding(2048, 768)
  )
  (text_embedding): Embedding(106339, 768)
  (t5_encoder): TransformerStack(
    (dropout): Dropout(p=0.1, inplace=False)
    (norm_out): LayerNorm()
    (layers): ModuleList(
      (0-5): 6 x TransformerBlock(
        (norm_self): LayerNorm()
        (self_attention): Attention(
          (qkv_net): Linear(in_features=768, out_features=2304, bias=False)
          (o_net): Linear(in_features=768, out_features=768, bias=False)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (layer_scale_self_attn): Identity()
        (norm_pos_ff): LayerNorm()
        (pos_ff): PositionwiseConvFF(
          (non_linearity): GELU(approximate='tanh')
          (proj): ConvNorm(
            (conv): Conv1d(768, 3072, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
          )
          (o_net): ConvNorm(
            (conv): Conv1d(3072, 768, kernel_size=(3,), stride=(1,), padding=(1,), bias=F

In [44]:
test_dataset = T5TTSDataset(
    dataset_meta={},
    sample_rate=model_cfg.sample_rate,
    min_duration=0.5,
    max_duration=20,
    codec_model_downsample_factor=model_cfg.codec_model_downsample_factor,
    bos_id=model.bos_id,
    eos_id=model.eos_id,
    context_audio_bos_id=model.context_audio_bos_id,
    context_audio_eos_id=model.context_audio_eos_id,
    audio_bos_id=model.audio_bos_id,
    audio_eos_id=model.audio_eos_id,
    num_audio_codebooks=model_cfg.num_audio_codebooks,
    prior_scaling_factor=None,
    load_cached_codes_if_available=True,
    dataset_type='test',
    tokenizer_config=None,
    load_16khz_audio=model.model_type == 'single_encoder_sv_tts',
    use_text_conditioning_tokenizer=model.use_text_conditioning_encoder,
    pad_context_text_to_max_duration=model.pad_context_text_to_max_duration,
    context_duration_min=model.cfg.get('context_duration_min', 5.0),
    context_duration_max=model.cfg.get('context_duration_max', 5.0),
)
test_dataset.text_tokenizer, test_dataset.text_conditioning_tokenizer = model._setup_tokenizers(model.cfg, mode='test')

[NeMo W 2025-01-13 23:43:50 experimental:26] `<class 'nemo.collections.tts.g2p.models.i18n_ipa.IpaG2p'>` is experimental and not ready for production yet. Use at your own risk.
[NeMo W 2025-01-13 23:43:50 i18n_ipa:124] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2025-01-13 23:43:50 experimental:26] `<class 'nemo.collections.common.tokenizers.text_to_speech.tts_tokenizers.IPATokenizer'>` is experimental and not ready for production yet. Use at your own risk.
[NeMo W 2025-01-13 23:43:50 experimental:26] `<class 'nemo.collections.tts.g2p.models.i18n_ipa.IpaG2p'>` is experimental and not ready for production yet. Use at your own risk.
[NeMo W 2025-01-13 23:43:53 i18n_ipa:124] apply_to_oov_word=None, This means that some of words will remain unchanged if they 

In [53]:
audio_dir = "/Data/RivaData/riva"
entry = {
    "audio_filepath": "Lindy/22khz/CMU_HAPPY/LINDY_CMU_HAPPY_000567.wav",
    "duration": 6.275604,
#     "text": "This is a very long sentence to test the speed of my text to speech synthesis model, let's see how long it takes to generate the audio. Adding more words to make it even longer. Haha.",
    "text": "S D S D Pass zero - zero Fail - zero to zero - zero - zero Cancelled" ,
    "speaker": "dummy",
    "context_text": "Speaker and Emotion: | Language:en Dataset:Riva Speaker:Rodney_CMU_HAPPY |",
#     "context_audio_filepath": "Rodney/22khz/DROP/RODNEY_DROP_000060.wav",
#     "context_audio_duration": 8.0,
}
data_sample = DatasetSample(
    dataset_name="sample",
    manifest_entry=entry,
    audio_dir=audio_dir,
    feature_dir=audio_dir,
    text=entry['text'],
    speaker=None,
    speaker_index=0
)
test_dataset.data_samples = [data_sample]

test_data_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,
    collate_fn=test_dataset.collate_fn,
    num_workers=0,
    shuffle=False
)


In [56]:
item_idx = 0
for bidx, batch in enumerate(test_data_loader):
    print("Processing batch {} out of {}".format(bidx, len(test_data_loader)))
    model.t5_decoder.reset_cache(use_cache=True)
    batch_cuda ={}
    for key in batch:
        if isinstance(batch[key], torch.Tensor):
            batch_cuda[key] = batch[key].cuda()
        else:
            batch_cuda[key] = batch[key]
    import time
    st = time.time()
    predicted_audio, predicted_audio_lens, _, _ = model.infer_batch(batch_cuda, max_decoder_steps=500, temperature=0.6, topk=80, use_cfg=True, cfg_scale=1.8)
    print("generation time", time.time() - st)
    for idx in range(predicted_audio.size(0)):
        predicted_audio_np = predicted_audio[idx].float().detach().cpu().numpy()
        predicted_audio_np = predicted_audio_np[:predicted_audio_lens[idx]]
        audio_path = os.path.join(out_dir, f"predicted_audio_{item_idx}.wav")
        sf.write(audio_path, predicted_audio_np, model.cfg.sample_rate)
        display(Audio(audio_path))
        item_idx += 1

Processing batch 0 out of 1
Decoding timestep 0
Decoding timestep 20
Decoding timestep 40
Decoding timestep 60
Decoding timestep 80
Decoding timestep 100
Decoding timestep 120
Decoding timestep 140
End detected for item 0 at timestep 150
All ends reached
generation time 2.93342661857605
